## stacking 集成算法-只有付费记录：最佳模型
基础学习器抽取特征，次级学习器融合。5折。
- 4 ExtraTreesRegressor
- 5 Gradient Boosting Regressor
- 6 RandomForestRegressor
- 7 BaggingRegressor

In [1]:
# 导入必要的工具包
import numpy as np #用于数值计算
import pandas as pd #用于数据表处理，数据文件读写
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn import ensemble

import matplotlib.pyplot as plt #用于绘图：分析结果的可视化。
#应该是设置浮点数的形式格式，小数点后三位
pd.set_option('display.float_format', lambda x: '%.4f' % x)
#这一行是干嘛的？ 设置 matplotlib 让绘制的图形出现在 Notebook 里而不是新窗口
%matplotlib inline

In [2]:
def get_train_val(val_id):
    '''获取训练集和校验集 
    val_id = 0,1,2 #当前校验集编号 '''
    print('val_id=%s'% (val_id))
    df_val = train_list[val_id]
    train_tmp = []

    for i in range(1, train_part_num):
        cur_id = (val_id + i) % 5
        print('cur_id=%s'% (cur_id))
        train_tmp.append(train_list[cur_id])
    
#     只生成一次，节省内存 
    df_train = pd.concat(train_tmp)
    print('df_train.shape=%s  df_val.shape=%s'% (df_train.shape, df_val.shape))
    return df_train, df_val

In [3]:
Y_Standard = True
path_train = '../../data/new/train=all-balance/'
path_test = '../../data/new/test/'

val_path = '../../data/val/payBase/' #保存校验集测试结果
val_name = '%sval=stacking-%s-%s.csv'
summit_path = '../../data/summit/payBase/' #保存提交文件，测试集测试结果 
out_name = '%ssummit=stacking-%s-%s.csv' # 生成的结果文件名称
base = 'payBase' #学习器级别 

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 指定读取数据类型，节省内存

In [4]:
col_int32 = ['user_id', 'register_time', 'wood_add_value', 'wood_reduce_value',
       'stone_add_value', 'stone_reduce_value', 'ivory_add_value',
       'ivory_reduce_value', 'meat_add_value', 'meat_reduce_value',
       'magic_add_value', 'magic_reduce_value', 'infantry_add_value',
       'infantry_reduce_value', 'cavalry_add_value', 'cavalry_reduce_value',
       'shaman_add_value', 'shaman_reduce_value', 'wound_infantry_add_value',
       'wound_infantry_reduce_value', 'wound_cavalry_add_value',
       'wound_cavalry_reduce_value', 'wound_shaman_add_value',
       'wound_shaman_reduce_value', 'general_acceleration_add_value',
       'general_acceleration_reduce_value', 'building_acceleration_add_value',
       'building_acceleration_reduce_value', 'reaserch_acceleration_add_value',
       'reaserch_acceleration_reduce_value', 'training_acceleration_add_value',
       'training_acceleration_reduce_value', 'treatment_acceleraion_add_value',
       'treatment_acceleration_reduce_value', 'bd_training_hut_level',
       'bd_healing_lodge_level', 'bd_stronghold_level',
       'bd_outpost_portal_level', 'bd_barrack_level',
       'bd_healing_spring_level', 'bd_dolmen_level', 'bd_guest_cavern_level',
       'bd_warehouse_level', 'bd_watchtower_level', 'bd_magic_coin_tree_level',
       'bd_hall_of_war_level', 'bd_market_level', 'bd_hero_gacha_level',
       'bd_hero_strengthen_level', 'bd_hero_pve_level', 'sr_scout_level', 
       'sr_training_speed_level', 'sr_infantry_tier_2_level',
       'sr_cavalry_tier_2_level', 'sr_shaman_tier_2_level',
       'sr_infantry_atk_level', 'sr_cavalry_atk_level', 'sr_shaman_atk_level',
       'sr_infantry_tier_3_level', 'sr_cavalry_tier_3_level',
       'sr_shaman_tier_3_level', 'sr_troop_defense_level',
       'sr_infantry_def_level', 'sr_cavalry_def_level', 'sr_shaman_def_level',
       'sr_infantry_hp_level', 'sr_cavalry_hp_level', 'sr_shaman_hp_level',
       'sr_infantry_tier_4_level', 'sr_cavalry_tier_4_level',
       'sr_shaman_tier_4_level', 'sr_troop_attack_level',
       'sr_construction_speed_level', 'sr_hide_storage_level',
       'sr_troop_consumption_level', 'sr_rss_a_prod_levell',
       'sr_rss_b_prod_level', 'sr_rss_c_prod_level', 'sr_rss_d_prod_level',
       'sr_rss_a_gather_level', 'sr_rss_b_gather_level',
       'sr_rss_c_gather_level', 'sr_rss_d_gather_level', 'sr_troop_load_level',
       'sr_rss_e_gather_level', 'sr_rss_e_prod_level',
       'sr_outpost_durability_level', 'sr_outpost_tier_2_level',
       'sr_healing_space_level', 'sr_gathering_hunter_buff_level',
       'sr_healing_speed_level', 'sr_outpost_tier_3_level',
       'sr_alliance_march_speed_level', 'sr_pvp_march_speed_level',
       'sr_gathering_march_speed_level', 'sr_outpost_tier_4_level',
       'sr_guest_troop_capacity_level', 'sr_march_size_level',
       'sr_rss_help_bonus_level', 'pvp_battle_count', 'pvp_lanch_count',
       'pvp_win_count', 'pve_battle_count', 'pve_lanch_count', 'pve_win_count',
       'pay_count']
col_float32 = ['avg_online_minutes', 'pay_price']

In [5]:
# 指定读取数据类型，节省内存
dtype_test = {}
for col in col_int32:
    dtype_test[col] = np.int32
for col in col_float32:
    dtype_test[col] = np.float32
# dtype_test

In [6]:
dtype_train = dtype_test.copy()
dtype_train[target] = np.float32
# dtype_train

## 读取数据，一次读取多次使用，不同组合 

In [7]:
train_list = []
for i in range(train_part_num):
    cur_id = i + 1
    train_list.append(pd.read_csv('%strain=pay-%s.csv' % (path_train, cur_id), dtype=dtype_train))
train_list[0].head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,563914,112,1007159,985765,88000,40000,3000,0,1290490,1006267,...,7,4,5,7,7,4,20.3333,1.9900,1,1.9900
1,1837494,106,50141850,50032020,42832440,42426724,23950932,19820632,46290384,37001236,...,1,0,0,21,21,19,105.1667,346.7900,21,346.7900
2,1775165,125,26860392,26935700,17661758,17542478,7595453,2596600,34394904,26972670,...,527,476,504,233,230,216,845.1667,7.9600,4,16.9100
3,3038897,118,1385557,354561,1200000,0,700000,0,2960605,280514,...,7,0,1,0,0,0,2.3333,0.9900,1,0.9900
4,285815,119,7800157,7032101,3436232,1893276,1478472,0,11743239,5703664,...,20,1,2,148,148,145,233.5000,1.9800,2,3.9700


## 5折交叉验证

In [8]:
# 根据校验集编号 0，1,2,3,4 ，获取训练集和校验集
df_train, df_val = get_train_val(0)

# 从原始数据中分离输入特征x和输出y
train_y = df_train[target].values
train_X = df_train.drop([target, user_id], axis = 1)
val_y = df_val[target].values
val_X = df_val.drop([target, user_id], axis = 1)

val_id=0
cur_id=1
cur_id=2
cur_id=3
cur_id=4
df_train.shape=(36790, 109)  df_val.shape=(9198, 109)


In [9]:
# 数据标准化：分别初始化对特征和目标值的标准化器 
ss_X = StandardScaler()
ss_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
train_X_ss = ss_X.fit_transform(train_X)
val_X_ss = ss_X.transform(val_X)

if Y_Standard:
    train_y_ss = ss_y.fit_transform(train_y.reshape(-1, 1))
    val_y_ss = ss_y.transform(val_y.reshape(-1, 1))

# 需要转换为一维数组 
train_y_ss_1d = train_y_ss.reshape(train_y_ss.shape[0])
val_y_ss_1d = val_y_ss.reshape(val_y_ss.shape[0])

In [10]:
def inverse_StandardScaler(predict):
    ''' 标准化数据还原  '''
    if Y_Standard:
        predict = ss_y.inverse_transform(predict)
        print('predict = ', predict)
    return predict

def generate_summit(predict):
    ''' 输出预测后的数据 '''
    testPredict = test.copy()
    testPredict[target] = predict
    testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

def generate_val_predict(predict):
    ''' 生成校验集预测后的数据 '''
    testPredict = df_val.copy()
    testPredict[target] = predict
    testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

## 模型训练
###  4. ExtraTreesRegressor

In [40]:
%%time
model = ensemble.ExtraTreesRegressor(n_estimators=35, min_samples_split=80, min_samples_leaf=8, n_jobs=-1)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 4min 6s

Wall time: 5.2 s


In [41]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss)
scoreVal = model.score(val_X_ss, val_y_ss)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss, predict_train) ** 0.5
predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss, predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=10, min_samples_split=80, min_samples_leaf=8 
# n_estimators=12, min_samples_split=80, min_samples_leaf=8
# n_estimators=14, min_samples_split=80, min_samples_leaf=8
# n_estimators=16, min_samples_split=80, min_samples_leaf=8 
# n_estimators=20, min_samples_split=80, min_samples_leaf=8
# n_estimators=30, min_samples_split=80, min_samples_leaf=8
# n_estimators=35, min_samples_split=80, min_samples_leaf=8
# The score: Train=0.5658510934333981  Val=0.4319487714342609
# The RMSE: Train=0.6588997622417258  Val=0.6433956917430198

The score: Train=0.5550744293751871  Val=0.4110179967425165
The RMSE: Train=0.6670273954810705  Val=0.9484005528839826


### 保存校验集测试结果

In [13]:
print('predict_val= ', predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'ExtraTreesRegressor'), index=False)
valPredict.head()

predict_val=  [-0.14429595  1.45171794 -0.05482073 ... -0.12758295 -0.07613492
 -0.12850887]
predict =  [  2.50654976 940.14620149  55.07220288 ...  12.32524614  42.55036256
  11.78127502]


,user_id,prediction_pay_price
0,563914,2.5065
1,1837494,940.1462
2,1775165,55.0722
3,3038897,1.1520
4,285815,11.6003


## 加载测试数据

In [14]:
test = pd.read_csv(path_test + 'tap_fun_test-fe.csv', dtype=dtype_test)
test_X = test.drop([user_id], axis = 1)
test_X_ss = ss_X.transform(test_X)
test.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14933,94,166415,138362,0,0,0,0,258522,90142,...,0,0,0,0,1,1,1,8.0000,0.0000,0
1,14934,94,10000,600,0,0,0,0,10000,400,...,0,0,0,0,0,0,0,0.1667,0.0000,0
2,14935,94,10000,3700,0,0,0,0,10000,2000,...,0,0,0,0,0,0,0,17.0000,0.0000,0
3,14936,94,210000,0,200000,0,200000,0,610000,0,...,0,0,0,0,0,0,0,1.6667,0.0000,0
4,14937,94,11500,3700,0,0,0,0,11000,2000,...,0,0,0,0,0,0,0,0.3333,0.0000,0


In [15]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)
# lasso_y_predict_test =  [0.4556051 0.4556051 0.4556051 ... 0.4556051 0.4556051 0.4556051]

model_y_predict_test =  [-0.08940467 -0.08464412 -0.06431621 ... -0.09359765 -0.05925197
 -0.05914361]
predict =  [34.75454194 37.55131051 49.49372031 ... 32.29121363 52.46890304
 52.53256428]


In [16]:
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'ExtraTreesRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,34.7545
1,14934,37.5513
2,14935,49.4937
3,14936,29.1128
4,14937,40.8236


## 5 GradientBoostingRegressor

In [17]:
%%time
model = ensemble.GradientBoostingRegressor(learning_rate=0.25, n_estimators=14, min_samples_split=80, min_samples_leaf=8, alpha=0.99)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 3min 2s 

Wall time: 1.83 s


In [18]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# learning_rate=0.1, n_estimators=50, min_samples_split=50, min_samples_leaf=10, alpha=0.8
# learning_rate=0.2, n_estimators=30, min_samples_split=50, min_samples_leaf=10, alpha=0.8
# learning_rate=0.2, n_estimators=20, min_samples_split=50, min_samples_leaf=10, alpha=0.8
# learning_rate=0.2, n_estimators=20, min_samples_split=50, min_samples_leaf=10, alpha=0.99
# learning_rate=0.25, n_estimators=16, min_samples_split=64, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=12, min_samples_split=64, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=12, min_samples_split=80, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=13, min_samples_split=80, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=14, min_samples_split=80, min_samples_leaf=8, alpha=0.99
# The score: Train=0.7474515268358191  Val=0.45818076230225474
# The RMSE: Train=0.5025420292903933  Val=0.8525285883738782

The score: Train=0.6520117252506124  Val=0.5294990584477097
The RMSE: Train=0.5899052938749633  Val=0.8476587209129436


### 保存校验集测试结果

In [19]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'GradientBoostingRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.10758627  1.14500888 -0.03316175 ... -0.10758627 -0.10758627
 -0.10758627]
predict =  [ 24.07306153 759.95818495  67.79660644 ...  24.07306153  24.07306153
  24.07306153]


,user_id,prediction_pay_price
0,563914,24.0731
1,1837494,759.9582
2,1775165,67.7966
3,3038897,24.0731
4,285815,24.0731


In [20]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.10758627 -0.10758627 -0.10758627 ... -0.10758627 -0.10758627
 -0.10758627]
predict =  [24.07306153 24.07306153 24.07306153 ... 24.07306153 24.07306153
 24.07306153]


In [21]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'GradientBoostingRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,24.0731
1,14934,24.0731
2,14935,24.0731
3,14936,24.0731
4,14937,24.0731


## 6 RandomForestRegressor

In [22]:
%%time
model = ensemble.RandomForestRegressor(n_estimators=16, min_samples_split=80, min_samples_leaf=8, n_jobs=-1)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 10min 2s 

Wall time: 6.97 s


In [23]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=14, min_samples_split=80, min_samples_leaf=8, min_impurity_decrease=0.1
# n_estimators=20, min_samples_split=80, min_samples_leaf=8, min_impurity_decrease=0.1
# n_estimators=20, min_samples_split=80, min_samples_leaf=8, min_impurity_decrease=0.0
# n_estimators=20, min_samples_split=100, min_samples_leaf=8
# n_estimators=16, min_samples_split=100, min_samples_leaf=8
# The score: Train=0.6193906706679225  Val=0.4333588747424302
# The RMSE: Train=0.6169354557150862  Val=0.8718379844283086 
# n_estimators=16, min_samples_split=80, min_samples_leaf=8
# The score: Train=0.6393206199636329  Val=0.35457588882649244
# The RMSE: Train=0.6005657993266247  Val=0.517339714712821

The score: Train=0.583956589556768  Val=0.49581719925796
The RMSE: Train=0.6450142538876716  Val=0.8774750957561084


### 保存校验集测试结果

In [24]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'RandomForestRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.1445315   1.69831279  0.32877925 ... -0.12928703 -0.0603818
 -0.1399901 ]
predict =  [   2.36816721 1085.0178158   280.43274633 ...   11.32411764   51.80514071
    5.03618722]


,user_id,prediction_pay_price
0,563914,2.3682
1,1837494,1085.0178
2,1775165,280.4327
3,3038897,2.2484
4,285815,6.0593


In [25]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.08238434 -0.07292834 -0.06258769 ... -0.02516248 -0.0492921
 -0.05108797]
predict =  [38.87890181 44.43419366 50.50920791 ... 72.49608479 58.32020776
 57.2651576 ]


In [26]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'RandomForestRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,38.8789
1,14934,44.4342
2,14935,50.5092
3,14936,39.3197
4,14937,53.4540


## 7 BaggingRegressor

In [27]:
%%time
model = ensemble.BaggingRegressor(n_jobs=2, n_estimators=20)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 9min 2s 

Wall time: 24.3 s


In [28]:
%%time
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=10 
# n_estimators=15 
# he score: Train=0.944126770370558  Val=0.35667035830687766
# The RMSE: Train=0.2363752321974746  Val=0.5473401373069637
# n_estimators=20 


The score: Train=0.9084004210793603  Val=0.5113913103519558
The RMSE: Train=0.30265421509921697  Val=0.8638162690790285
Wall time: 4.5 s


### 保存校验集测试结果

In [29]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'BaggingRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.14500466  2.11534174  0.15957826 ... -0.12965373 -0.01657653
 -0.13261034]
predict =  [   2.09019103 1330.0174898   181.02912418 ...   11.10868509   77.54022964
    9.37170776]


,user_id,prediction_pay_price
0,563914,2.0902
1,1837494,1330.0175
2,1775165,181.0291
3,3038897,2.7386
4,285815,13.0659


In [30]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.13710695 -0.13593721  0.04477152 ... -0.07671938 -0.09576869
 -0.1072685 ]
predict =  [  6.73000233   7.41721569 113.58149923 ...  42.20699965  31.01575206
  24.2597502 ]


In [31]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'BaggingRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,6.7300
1,14934,7.4172
2,14935,113.5815
3,14936,23.5250
4,14937,122.9885


## 7-1 BaggingRegressor的代码最佳示例
https://www.helplib.com/Python-Module-Examples/article_66154